# Считает длину аудио

In [1]:
! pip install mutagen


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
import mutagen 
from mutagen.wave import WAVE
import pandas as pd
from tqdm import tqdm

In [3]:
# function to convert the information into 
# some readable format 
def audio_duration(length): 
	hours = length // 3600 # calculate in hours 
	length %= 3600
	mins = length // 60 # calculate in minutes 
	length %= 60
	seconds = length # calculate in seconds 

	return hours, mins, seconds # returns the duration 

In [9]:
# Create a WAVE object 
# Specify the directory address of your wavpack file
lengths = []

filenames = os.listdir('../audio')
wav_filenames = [filename for filename in filenames if filename.endswith('wav')]


for filename in tqdm(wav_filenames):
    audio = WAVE("../audio/"+filename)
    audio_info = audio.info
    lengths.append(int(audio_info.length))

100%|███████████████████████████████████████| 137/137 [00:00<00:00, 1165.35it/s]


# Статистика для корпуса Пакендорф

In [11]:
wav_df = pd.DataFrame()
wav_df['filename'] = wav_filenames
wav_df['length'] = lengths
wav_df['place'] = wav_df['filename'].apply(lambda x: x.split('__')[0])

In [12]:
wav_df.head()

,filename,length,place
0,Sebjan-Küöl__Narratives__Personal Narratives...,595,Sebjan-Küöl
1,Bystraja District__Narratives__Personal Narrat...,128,Bystraja District
2,Bystraja District__Phonetic recordings__VIA__V...,310,Bystraja District
3,Bystraja District__Narratives__Personal Narrat...,51,Bystraja District
4,Topolinoe__Narratives__Personal Narratives__To...,210,Topolinoe


In [14]:
wav_df['place'] = wav_df['place'].apply(lambda x: x if x != 'Sebjan-Küöl' else 'Sebjan-Küöl')

In [21]:
# минут
wav_df.groupby('place')['length'].sum() / 60

place
Bystraja District    1042.550000
Sebjan-Küöl           269.566667
Topolinoe             372.250000
Name: length, dtype: float64

In [17]:
# часов
wav_df.groupby('place')['length'].sum() / 60 / 60

place
Bystraja District    17.375833
Sebjan-Küöl           4.492778
Topolinoe             6.204167
Name: length, dtype: float64

In [25]:
# файлов
wav_df['place'].value_counts()

place
Bystraja District    78
Sebjan-Küöl          31
Topolinoe            28
Name: count, dtype: int64

### Не будем считать фонетические пары

In [19]:
wav_df['phonetic'] = wav_df['filename'].apply(lambda x: True if 'Phonetic' in x else False)
wav_df.head()

,filename,length,place,phonetic
0,Sebjan-Küöl__Narratives__Personal Narratives...,595,Sebjan-Küöl,False
1,Bystraja District__Narratives__Personal Narrat...,128,Bystraja District,False
2,Bystraja District__Phonetic recordings__VIA__V...,310,Bystraja District,True
3,Bystraja District__Narratives__Personal Narrat...,51,Bystraja District,False
4,Topolinoe__Narratives__Personal Narratives__To...,210,Topolinoe,False


In [22]:
# минут
wav_df[wav_df['phonetic'] == False].groupby('place')['length'].sum() / 60

place
Bystraja District    242.416667
Sebjan-Küöl          269.566667
Topolinoe            372.250000
Name: length, dtype: float64

In [23]:
# часов
wav_df[wav_df['phonetic'] == False].groupby('place')['length'].sum() / 60 / 60

place
Bystraja District    4.040278
Sebjan-Küöl          4.492778
Topolinoe            6.204167
Name: length, dtype: float64

In [26]:
# файлов
wav_df[wav_df['phonetic'] == False]['place'].value_counts()

place
Sebjan-Küöl          31
Topolinoe            28
Bystraja District    26
Name: count, dtype: int64